In [1]:
# import Pkg; Pkg.add("POMDPPolicies")
# Pkg.add("POMDPModelTools")
# Pkg.add("Distributions")
# Pkg.add("Combinatorics")
# Pkg.add("StaticArrays")
# Pkg.add("FileIO")
# Pkg.add("JLD2")
# Pkg.add("TickTock")
# Pkg.add("POMDPTools")
# Pkg.add("D3Trees")
# Pkg.add("Random")
# Pkg.add("DataFrames")
# Pkg.add("CSV")

In [2]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [3]:
Number_level = 4;


gamma=0.95
Transition_matrix = [
    0.8571 0.1429 0.0    0.0;
    0.0    0.8571 0.1429 0.0;
    0.0    0.0    0.8    0.2;
    0.0    0.0    0.0    1.0
]
Observation_matrix=[
    0.8 0.2 0.0 0.0;
    0.1 0.8 0.1 0.0;
    0.0 0.2 0.8 0.0;
    0.0 0.0 0.0 1.0
]
# Observation_matrix=[
#     0.65  0.25 0.1   0.0;
#     0.175 0.65 0.175 0.0;
#     0.1   0.25 0.65  0.0;
#     0.0   0.0  0.0   1.0
# ]

States=Array((1:Number_level))
Observations=Array((1:Number_level))

Initialstates = zeros(Number_level)
Initialstates[1]=1.0
Initialstate = SparseCat(States, Initialstates)

# Initialobss = zeros(Number_level)
# Initialobss[1]=1.0
# Initialobs = SparseCat(States, Initialobss)
Initialobs = SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [4]:
NumberUnits=20
limit=12
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

## Observation function

In [5]:
function observation_function(s)
    return SparseCat(Observations, Observation_matrix[s,:])
end
# obs_std=[1,1.2,1.5]
# obs_mean=[0.5,1,1.5]

# function observation_function(s)
#     if s==Number_level
#         return Deterministic(Number_level)
#     else
#         cont_dist=Normal(obs_mean[s],obs_std[s])
#         probs=zeros(Number_level)
#         for i in 1:(Number_level-1)
#             if i==1
#                 probs[i]=cdf(cont_dist,0.75)
#             elseif i==(Number_level-1)
#                 probs[i]=1-cdf(cont_dist,1.25)
#             else
#                 probs[i]=cdf(cont_dist,i+0.25)-cdf(cont_dist,i-0.25)
#             end
#         end
#         probs ./= sum(probs)
#         return SparseCat(1:(Number_level),probs)
#     end
# end


observation_function (generic function with 1 method)

## Belief updater

In [6]:
function update_belief(b,s_b,a,o) 
    # NOTE: 
    # in input, b is belief from last epoch (before a)
    # a is the last action we took
    # o is the new observation after a
    updated_belief = zeros(Number_level)
    if o==Number_level
        updated_belief[Number_level]=1.0
        return updated_belief
    end
    
    if s_b >= a
    # We assume perfect maintenance and it is know to decision maker(belief updater)
        for sp in 1: Number_level
            #p_obs=pdf(SparseCat(Observations, Observation_matrix[sp,:]),o)
            p_obs=pdf(observation_function(sp),o)
            p_trans = pdf(SparseCat(States,Transition_matrix[1,:]),sp) 
            updated_belief[sp] = p_obs * p_trans
        end
        updated_belief ./= sum(updated_belief)
        return updated_belief
    end
    
    for sp in 1:Number_level
        #p_obs=pdf(SparseCat(Observations, Observation_matrix[sp,:]),o)
        p_obs=pdf(observation_function(sp),o)
        p_trans = 0.0
        for s in 1:Number_level
            p_trans += pdf(SparseCat(States,Transition_matrix[s,:]),sp) * b[s]
        end
        updated_belief[sp] = p_obs * p_trans
    end
        # Normalize the updated belief state to ensure it sums up to 1
        updated_belief ./= sum(updated_belief)
    return updated_belief
end

update_belief (generic function with 1 method)

In [7]:
NumberUnits = 10
belief = [[0.8, 0.2, 0.0, 0.0] for j in 1:NumberUnits]
b = belief[1,:]

1-element Vector{Vector{Float64}}:
 [0.8, 0.2, 0.0, 0.0]

In [8]:
b = belief[1]

4-element Vector{Float64}:
 0.8
 0.2
 0.0
 0.0

In [9]:
p_trans = 0.0
s = 1
sp = 2

p_trans += pdf(SparseCat(States,Transition_matrix[s,:]),sp) * b[s]

0.11432

## Define POMDP

In [10]:
# s is the system-level state vector
#s,r,o,true_a=approximate(s,s_belief,local_a,234) 
# Approximating the solution by taking the state of highest belief as the true state
function approximate(s,s_belief,a,rng=234)
    r=0.0
    prevent_repair= false
    failed_components=count(x->x==Number_level,s_belief)
    if failed_components >= limit
        r += system_penalty
        # if a==Number_level +1
        #     a = Number_level
        # end
    end

    for i in 1:NumberUnits
        if s_belief[i] >= a 
            # Maintenance action is decided based on belief, not true state
            # since we don't know true states when making decision
            if s_belief[i] == Number_level
                r += failure_penalty
                if prevent_repair == false
                    r += setup_cost
                    prevent_repair = true
                end
            else
                r += maintenance_penalty
                if prevent_repair == false
                    r += setup_cost
                    prevent_repair = true
                end
            end
            s[i]=rand(crd[1])
            r += normal_operation
        else
            s[i] = rand(crd[s[i]])
            r = r+normal_operation
        end
    end

    obs=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        #obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
        obs_distribution = observation_function(s[i])
        obs[i] = rand(obs_distribution)
    end

    return (s=s,r=r,o=obs,a=a)
end

approximate (generic function with 2 methods)

In [11]:
# # Generate random positive numbers and normalize each row
# function generate_prob_matrix(rows, cols)
#     # Create random positive numbers
#     matrix = rand(rows, cols)
    
#     # Normalize each row to sum to 1
#     normalized = matrix ./ sum(matrix, dims=2)
    
#     return normalized
# end

# # Generate 20x4 matrix
# prob_matrix = generate_prob_matrix(20, 4)

In [12]:
# global initialb= zeros(20, 4)  
# global initialb[:, 1] .= 1.0
# s_belief = [argmax(row) for row in eachrow(initialb)]

In [13]:
# belief[1,:]=[0.3,0.3,0.4,0]
# true_a=3
# o[1]=2

In [14]:
# update_belief([0.3,0.3,0.4,0], 3,3, 1)

In [15]:
function findnN()
    rewards_nN = zeros(Number_level,Number_level);
    rewards_nN = rewards_nN.+(-100000000.0);
    temp_rewards = zeros(repetition,1);
    # global rewards_nN_std = zeros(Number_level,Number_level);
    # global rewards_nN_std = rewards_nN_std.+(-100000000.0);
    for N in 2:(Number_level)
        for n in 1:N
            # temp_rewards = zeros(repetition,1)
            Threads.@threads  for j in 1:repetition
            #for j in 1:repetition
                s=repeat(1:1,NumberUnits)
                # o=s 
                o = rand(Initialobs, NumberUnits)
                # belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
                belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
                R_sequence=[]
                for k in 1:simsteps
                    # s_belief = [argmax(row) for row in eachrow(belief)]
                    s_belief = argmax.(belief)
                    local_a = Number_level +1; # when local_a = 0, no repair unless system fails
                    for i in 1:NumberUnits
                    # if there is any unit reaching or exceeding threshold N, 
                    # maintain all the units reaching or exceeding threshold n.
                        
                        if s_belief[i]>= N
                            local_a=n
                        end
                    end
                    true_a=local_a
                    # s is the system-level state vector
                    s,r,o,true_a=approximate(s,s_belief,local_a,234) 
                    for i in 1: NumberUnits
                        belief[i]=update_belief(belief[i], s_belief[i],true_a, o[i])
                    end
                    append!(R_sequence,r)
                end
                R_discounted=0
                for k in 1:simsteps
                    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
                end
                temp_rewards[j]=R_discounted
            end
            rewards_nN[n,N] = mean(temp_rewards)
            #rewards_nN_std[n,N] = std(temp_rewards)
        end
    end
    # This max_reward is actually wrong in POMDP
    # The (n, N) obtained should be calculated based on true state
    (max_reward,nN)=findmax(rewards_nN)
    
    #println("Max rewards of (n, N)-policy is", max_reward, "n is ", nN[1], "N is ", nN[2])
    #return max_reward, rewards_nN_std[nN], nN
    return nN
end

findnN (generic function with 1 method)

In [16]:
# function generative(s,s_belief,a,rng=234)
#     r=0.0
#     prevent_repair= false
#     failed_components=count(x->x==Number_level,s_belief)
#     if failed_components >= limit
#         r += system_penalty
#         if a==Number_level +1
#             a = Number_level
#         end
#     end

#     for i in 1:NumberUnits
#         if s_belief[i] >= a
#             if s[i] == Number_level
#                 r += failure_penalty
#                 if prevent_repair == false
#                     r += setup_cost
#                     prevent_repair = true
#                 end
#             else
#                 r += maintenance_penalty
#                 if prevent_repair == false
#                     r += setup_cost
#                     prevent_repair = true
#                 end
#             end
#             s[i]=rand(crd[1])
#             r += normal_operation
#         else
#             s[i] = rand(crd[s[i]])
#             r = r+normal_operation
#         end
#     end

#     obs=repeat(1:1,NumberUnits)
#     for i in 1:NumberUnits
#         #obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
#         obs_distribution = observation_function(s[i])
#         obs[i] = rand(obs_distribution)
#     end

#     return (s=s,r=r,o=obs,a=a)
# end

In [17]:
function generative(s,s_belief,a,rng=234)
    r=0.0
    # prevent_repair= false
    # failed_components=count(x->x==Number_level,s_belief)
    # if failed_components >= limit
    #     #r += system_penalty
    #     if a==Number_level +1
    #         a = Number_level
    #     end
    # end

    if a <= Number_level
         r += setup_cost
    end

    true_failed_components=count(x->x==Number_level,s)
    if true_failed_components >= limit
        r += system_penalty
    end

    for i in 1:NumberUnits
        if s_belief[i] >= a
            if s[i] == Number_level
                r += failure_penalty
            else
                r += maintenance_penalty
            end
            s[i]=rand(crd[1])
            r += normal_operation
        else
            s[i] = rand(crd[s[i]])
            r = r+normal_operation
        end
    end

    obs=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        #obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
        obs_distribution = observation_function(s[i])
        obs[i] = rand(obs_distribution)
    end

    return (s=s,r=r,o=obs)
end

generative (generic function with 2 methods)

In [18]:
# Define the fixed K values for each unit count
function get_fixed_K(units)
    if units == 60
        return 18
    elseif units == 50
        return 18
    elseif units == 40
        return 18
    elseif units == 30
        return 14
    elseif units == 25
        return 10  # Assuming a reasonable value for 25
    elseif units == 20
        return 10   # Assuming a reasonable value for 20
    else
        return convert(Int64, floor((units+1)/4))  # Fallback for any other values
    end
end

get_fixed_K (generic function with 1 method)

In [19]:
using DataFrames

observation_matrices = [
    # Low Uncertainty
    [
        0.9  0.1  0.0  0.0;
        0.05 0.9  0.05 0.0;
        0.0  0.1  0.9  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Original Uncertainty
    [
        0.8  0.2  0.0  0.0;
        0.1  0.8  0.1  0.0;
        0.0  0.2  0.8  0.0;
        0.0  0.0  0.0  1.0
    ],
    # High Uncertainty
    [
        0.7  0.3  0.0  0.0;
        0.15 0.7  0.15 0.0;
        0.0  0.3  0.7  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Very High Uncertainty
    [
        0.6  0.4  0.0  0.0;
        0.2  0.6  0.2  0.0;
        0.0  0.4  0.6  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Extreme Uncertainty
    [
        0.5  0.5  0.0  0.0;
        0.25 0.5  0.25 0.0;
        0.0  0.5  0.5  0.0;
        0.0  0.0  0.0  1.0
    ]
]

# Labels for the observation matrices (useful for output and plotting)
uncertainty_labels = ["Low", "Original", "High", "Very High", "Extreme"]

# Modified DataFrame with an additional column for uncertainty level

df= DataFrame(uncertainty=[],u=[],K=[],n=[],s=[],m=[],f=[],p=[],mean=[],std=[],nN=[])

cost=[ 
    [0  -2400   -100    -1200 -2000]
] 


for units in [20, 25, 30, 40, 50, 60]

    global limit = get_fixed_K(units)
    global K = limit
          
        
        global NumberUnits=convert(Int64,units)
        global crd = Array{Categorical}(undef,Number_level)
        for i in 1:Number_level
            global crd[i] = Categorical(Transition_matrix[i,:]);
        end
            
        # global initialb= zeros(NumberUnits, Number_level)  
        # global initialb[:, 1] .= 1.0
        for c in cost
            for obs_idx in 1:length(observation_matrices)
                global Observation_matrix = observation_matrices[obs_idx]
                global uncertainty_level = uncertainty_labels[obs_idx]
            tick()
            println(units)  
            print("K : ")
            println(K)
            global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
            println(c) 

            println("Uncertainty Level: ")
                print(uncertainty_level)
            global NumberUnits=units

            global simsteps = 100;
            global repetition = 10000;
            global rewards_nN = zeros(Number_level,Number_level);
            global rewards_nN = rewards_nN.+(-100000000.0);
            global rewards_nN_std = zeros(Number_level,Number_level);
            global rewards_nN_std = rewards_nN_std.+(-100000000.0);

            global discount_factor =gamma
            global temp_rewards = zeros(repetition,1);
            # Looking for optimal (n, N)
            println("Finding best nN policy...")
            nN =findnN()
            n=nN[1]
            N=nN[2]
            
            trials=10000;
            results=zeros(trials)*0.1
            # cnt_maintenance = zeros(trials)
            rewards=0.0
            rewards_std=0.0
            Threads.@threads for h in 1:trials
                s=repeat(1:1,NumberUnits)
                # o=s 
                o = rand(Initialobs, NumberUnits)
                # belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
                belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
                R_sequence=[]
              
                
                # maintenance_count = 0
                for k in 1:simsteps
                    # s_belief = [argmax(row) for row in eachrow(belief)]
                    s_belief = argmax.(belief)
                    # if h==1
                    #     println("The belief vector is ", belief)
                    #     println("====================================================")
                    #     println("The maximum likelihood state vector is", s_belief)
                    # end
                    local_a = Number_level +1; 
                    for i in 1:NumberUnits
                    # If there is any unit reaching or exceeding threshold N, 
                    # maintain all the units reaching or exceeding threshold n,
                    # otherwise, no maintenance.
                    # As a result, when system fails, there are components at failure level,
                    # so N must be reached.
                        if s_belief[i]>= N
                            local_a=n
                        end
                    end
                    # true_a=local_a
                    # s is the system-level state vector
                    # if local_a <= Number_level
                    #     # println("maintain")
                    #     maintenance_count += 1
                    # end
                    s,r,o=generative(s,s_belief,local_a,234) 
                    for i in 1: NumberUnits
                        # modified: belief[i,:] to belief[i]
                        belief[i]=update_belief(belief[i], s_belief[i],local_a, o[i])

                    end
                    append!(R_sequence,r)
                end
                R_discounted=0.0
                for k in 1:simsteps
                    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
                end
                # cnt_maintenance[h]=maintenance_count
                results[h]=R_discounted
            end
            rewards=mean(results)
            rewards_std=std(results)
            # N_maintenance = mean(cnt_maintenance)
            println("Rewards of (n, N)-policy is", rewards, "n is ", nN[1], "N is ", nN[2])
            # println("Number of maintenance times is ", N_maintenance)
            println(std(results))

            pushfirst!(df,[uncertainty_level,units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,rewards,std, nN])
        end
    end
end

20
K : 10


[ Info:  started timer at: 2025-04-11T15:03:41.126


[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
LowFinding best nN policy...
Rewards of (n, N)-policy is-14647.50170039497n is 2N is 4
1791.391551094614
20
K : 10
[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
OriginalFinding best nN policy...


[ Info:  started timer at: 2025-04-11T15:06:11.127


Rewards of (n, N)-policy is-15242.82627913775n is 2N is 4
1944.6378854449117
20
K : 10
[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
HighFinding best nN policy...


...

Rewards of (n, N)-policy is-31382.304290994573n is 2N is 4
2511.6267621775883
60
K : 18
[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
Very HighFinding best nN policy...


[ Info:  started timer at: 2025-04-11T17:00:08.509


Rewards of (n, N)-policy is-33071.99913796927n is 2N is 4
2778.782179196532
60
K : 18
[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
ExtremeFinding best nN policy...


[ Info:  started timer at: 2025-04-11T17:06:49.065


Rewards of (n, N)-policy is-35884.507394546425n is 2N is 4
3257.4233364197685


In [20]:
df

Row,uncertainty,u,K,n,s,m,f,p,mean,std,nN
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Extreme,60,18,0,-2400,-100,-1200,-2000,-35884.5,std,"CartesianIndex(2, 4)"
2,Very High,60,18,0,-2400,-100,-1200,-2000,-33072.0,std,"CartesianIndex(2, 4)"
3,High,60,18,0,-2400,-100,-1200,-2000,-31382.3,std,"CartesianIndex(2, 4)"
4,Original,60,18,0,-2400,-100,-1200,-2000,-30242.1,std,"CartesianIndex(2, 4)"
5,Low,60,18,0,-2400,-100,-1200,-2000,-28893.9,std,"CartesianIndex(2, 4)"
6,Extreme,50,18,0,-2400,-100,-1200,-2000,-31725.3,std,"CartesianIndex(2, 4)"
7,Very High,50,18,0,-2400,-100,-1200,-2000,-29201.1,std,"CartesianIndex(2, 4)"
8,High,50,18,0,-2400,-100,-1200,-2000,-27697.3,std,"CartesianIndex(2, 4)"
9,Original,50,18,0,-2400,-100,-1200,-2000,-26748.5,std,"CartesianIndex(2, 4)"


In [21]:
using CSV
CSV.write("(n,N).csv", df)

"(n,N).csv"